# Script to make list of peptides for input to R to make protein targets

In [1]:
from pyteomics import mgf
from pyteomics import mzxml
import pandas as pd
import numpy as np
from pyteomics import mass
import re
import pickle
import timeit
import time
import matplotlib.pyplot as plt

In [2]:
### makes a dict of scan:CV value pairs for lookup of the CV
tmp_cv_dict = {}
with mzxml.read("P:/JGM_DI2A/SILAC/20190411_DI2A_1to16_n1b.mzXML") as spectra:
    for x in spectra:
        tmp_cv_dict[x['num']] = x['compensationVoltage']
        #tmp_cv_list.append(x['compensationVoltage'])

In [3]:
# read the search results and sort by cosine score
# read+sort by cosine
df = pd.read_csv("P:/JGM_DI2A/SILAC/2da10ppm_1to16_n1b.txt", sep="\t").sort_values('cosine', ascending=False)
# keep only the row that contains the best cosine score for a peptide
df = df.loc[df.groupby('Peptide').cosine.idxmax()].sort_values('cosine', ascending=False)
df = df.reset_index()

In [4]:
# keep only the lines where peptide ended with 'K' or 'R'
endsrk = [df.loc[i] for i in range(0, len(df)) if np.any([df.loc[i]['Peptide'][-1:] =='R',df.loc[i]['Peptide'][-1:] =='K']) ]


In [5]:
# loop to compute the number of decoys and return the new df
### keep the decoys this time to compute protein FDR later

fdrlevel = 0.01
fdr=0
sortedpepdf = pd.DataFrame(endsrk)

decoylines = [i for i, e in enumerate(sortedpepdf.Name.tolist()) if e == 'DECOY_null']
ndecoys = len(decoylines)
nlines=len(sortedpepdf)
lastdecoy = []

if(ndecoys==0):
    lastdecoy.append()
else:
    for i in range(0, ndecoys):
        fdr = (i+1)/decoylines[i]
        if(fdr>fdrlevel):
            lastdecoy.append(i)
if(len(lastdecoy)==0):  # handles when there are not enough decoy hits
    #sortedpepdf = sortedpepdf.drop(sortedpepdf.index[decoylines])
    print('not enough decoy hits, accept all '+str(nlines-ndecoys)+' peptides @ FDR='+str(ndecoys/nlines))
    
# need to edit this from the R version
#if(len(lastdecoy)>=1 and (min(lastdecoy)-1) !=0):  ### handle when the FDR is achieved
    #i=min(lastdecoy)-1
    #fdr<-i/decoylines[i]
    #print(paste("fdr", round(length(decoylines[1:i])/decoylines[i], digits = 4)))
    #print("score cutoff")
    #print(t.first[decoylines[i],"cosine"])
    #print(paste("peptide hits=", decoylines[i]-i))
    #pep.output<-t.first[1:max(decoylines),]
    
if(len(lastdecoy)>=1 and (min(lastdecoy)-1) ==0): # handle when there are too many decoys
    print('FDR over '+str(fdrlevel))

not enough decoy hits, accept all 2248 peptides @ FDR=0.005749668288367979


In [6]:
sortedpepdf = sortedpepdf.reset_index()

In [7]:
# from filtered dataframe, take scan# and check what compensation voltage this uses and add to the dataframe
sortedpepdf['CV'] = [tmp_cv_dict[str(x)] for x in sortedpepdf['Scan#'].tolist()]

In [8]:
# get the FAIMS compensation voltages for the productive scan
# make dictionary with peptide sequence as key and list of [scan, cv] as value

sortedpepdf.head(10)

,level_0,index,#File,Scan#,Mz,z,Peptide,Mz.1,z.1,cosine,...,stat11,stat12,stat13,stat14,stat15,stat16,stat17,stat18,stat19,CV
0,0,3112,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,2420,420.0,2,APIIAVTR,420.77,2,0.976826,...,34.0,82.0,21.0,0.0,0.936531,0.964773,2.925132,NaN,NaN,-70.0
1,1,166,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,403,803.0,2,ALTGHLEEVVLALLK,803.48,2,0.973920,...,8.0,13.0,4.0,0.0,0.979685,0.948266,0.161017,NaN,NaN,-30.0
2,2,123,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,375,775.0,2,GTDVNVFNTILTTR,775.91,2,0.972525,...,2.0,21.0,7.0,0.0,0.264671,0.970944,0.220307,NaN,NaN,-30.0
3,3,85,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,333,733.0,2,S+42.01057DAAVDTSSEITTK,733.85,2,0.970295,...,10.0,19.0,5.0,0.0,0.053764,0.772867,0.807127,NaN,NaN,-30.0
4,4,744,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,785,585.0,2,ILLAELEQLK,585.36,2,0.970021,...,16.0,51.0,4.0,0.0,0.177908,0.808319,0.036934,NaN,NaN,-40.0
5,5,6798,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,5790,790.0,2,RVIISAPSADAPMFVMGVNHEK,790.41,3,0.967412,...,26.0,68.0,28.0,0.0,0.291999,0.966274,3.928140,NaN,NaN,-60.0
6,6,4919,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,4444,644.0,2,PMFIVNTNVPR,644.35,2,0.965606,...,27.0,66.0,25.0,0.0,0.266585,0.918693,3.397003,NaN,NaN,-40.0
7,7,6547,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,5569,569.0,2,ESYSVYVYK,569.28,2,0.964394,...,26.0,57.0,20.0,0.0,0.404750,0.956392,3.079535,NaN,NaN,-60.0
8,8,564,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,734,534.0,2,ESTLHLVLR,534.31,2,0.964187,...,19.0,43.0,3.0,0.0,0.455227,0.911493,0.055857,NaN,NaN,-40.0
9,9,4380,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,4260,460.0,2,GLFIIDDK,460.76,2,0.963987,...,30.0,65.0,19.0,0.0,0.331369,0.629725,1.933132,NaN,NaN,-40.0


### Determine what the heavy masses should be based on the # arginine and lysine, add column to df

In [11]:
def getHeavyPrec(sequence, mz, z):
    hK=8.014199 ## mass of heavy lysine
    hR=10.00827 ## mass of heavy arg
    ### fix masses and get proton mass w/o electron

    # clean sequence to remove mod masses and change oxMet to 'm'
    nR = len(sequence) - len(re.sub('R','', sequence))
    nK = len(sequence) - len(re.sub('K','', sequence))

    heavymz = mz + (nK*hK)/z + (nR*hR)/z
    heavymz = round(heavymz, ndigits = 2)
    return heavymz
    

In [12]:
sortedpepdf['Heavymz']=[getHeavyPrec(sortedpepdf['Peptide'][x], sortedpepdf['Mz.1'][x], sortedpepdf['z.1'][x]) for x in range(0, len(sortedpepdf))]
    

In [13]:
sortedpepdf.head()

,level_0,index,#File,Scan#,Mz,z,Peptide,Mz.1,z.1,cosine,...,stat12,stat13,stat14,stat15,stat16,stat17,stat18,stat19,CV,Heavymz
0,0,3112,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,2420,420.0,2,APIIAVTR,420.77,2,0.976826,...,82.0,21.0,0.0,0.936531,0.964773,2.925132,NaN,NaN,-70.0,425.77
1,1,166,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,403,803.0,2,ALTGHLEEVVLALLK,803.48,2,0.973920,...,13.0,4.0,0.0,0.979685,0.948266,0.161017,NaN,NaN,-30.0,807.49
2,2,123,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,375,775.0,2,GTDVNVFNTILTTR,775.91,2,0.972525,...,21.0,7.0,0.0,0.264671,0.970944,0.220307,NaN,NaN,-30.0,780.91
3,3,85,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,333,733.0,2,S+42.01057DAAVDTSSEITTK,733.85,2,0.970295,...,19.0,5.0,0.0,0.053764,0.772867,0.807127,NaN,NaN,-30.0,737.86
4,4,744,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,785,585.0,2,ILLAELEQLK,585.36,2,0.970021,...,51.0,4.0,0.0,0.177908,0.808319,0.036934,NaN,NaN,-40.0,589.37


### have light and heavy precursor, need to add the 3 most abundant matched y-ions

In [14]:
#sortedpepdf.to_csv("exact_targets.txt", sep="\t")

In [16]:
### now open file for quant
# make dict the msx[1] isolation centers for each scan and the
precmzlist= []
mz_plus_cv_dict = {}

with mzxml.read("P:/JGM_DI2A/SILAC/20190412_DI2A_1to1_tMS2_n3.mzXML") as spectra:
    for x in spectra:
        tmpscannum = x['num']
        tmplightisolationcenter = x['precursorMz'][0]['precursorMz']
        tmpcv = x['compensationVoltage']
        precmzlist.append(tmplightisolationcenter)
        if str(tmplightisolationcenter)+','+str(tmpcv) not in mz_plus_cv_dict.keys():
            mz_plus_cv_dict[str(tmplightisolationcenter)+','+str(tmpcv)] = [int(tmpscannum)]
        else:
            mz_plus_cv_dict[str(tmplightisolationcenter)+','+str(tmpcv)].append(int(tmpscannum))

### Function to find the precursor isolation target closest to the peptide precursor mass

In [17]:
# make new column in dataframe that has the closest light ms1 isolation center
def find_nearest(array,value):
    idx = (np.abs(array-np.float_(value))).argmin()
    return array[idx]

# then use the function to add column to the dataframe
sortedpepdf['quantprecursor'] = [find_nearest(precmzlist, x) for x in sortedpepdf['Mz.1'].tolist()]

In [18]:
# get the quant scans using the dictionary with 'precmz,CV' keys
quantscans = []
for x in range(0, len(sortedpepdf)):
    quantscans.append(mz_plus_cv_dict[str(sortedpepdf['quantprecursor'].tolist()[x]) +',' +str(sortedpepdf['CV'].tolist()[x])])

In [19]:
sortedpepdf['quantscans'] = quantscans

In [20]:
allpeptides = sortedpepdf['Peptide'].tolist()

### Read the spectral library mgf file

In [21]:
speclib = mgf.read('P:/JGM_DI2A/MSPLIT-DIAv1.0/human.faims.fixed.decoy.mgf')

### Function to return the 3 most intense heavy and light y ions from library spectra for quant

In [23]:
# function doesn't care about the observed masses, just looks in spectral library
def getQuantFragsFromLib(sequence, specname, libfileobj):
    '''
    PURPOSE: determine y-ion fragments to use for SILAC quantification
    
    INPUT: spectral library mgf object used with MSPLIT-DIA
    
    OUTPUT: pandas dataframe of the top 3 y-ion sequence fragments and their light and heavy yion fragments
    '''
    libmzarray = libfileobj.get_by_id(specname)['m/z array'].tolist()
    libintarray = libfileobj.get_by_id(specname)['intensity array'].tolist()
    hK=8.014199 ## mass of heavy lysine
    hR=10.00827 ## mass of heavy arg
    ### fix masses and get proton mass w/o electron
    customAAcomp = dict(mass.std_aa_comp)
    customAAcomp['C'] = mass.Composition({'H':8, 'C':5, 'S':1,'O':2, 'N': 2}) ## add carbamidomethyl to the modifications
    customAAcomp['m'] = mass.Composition({'H': 9, 'C': 5, 'S': 1, 'O': 2, 'N': 1}) ## add oxidized methionine
    protonmass = mass.calculate_mass(formula='H')-0.00054858 ## mass of hydrogen minus an electron
    # clean sequence to remove mod masses and change oxMet to 'm'
    sequence = re.sub('\+42.01057','', sequence)
    sequence = re.sub('\+57.0215','', sequence)
    sequence = re.sub('M\+15.9949','m', sequence)
    # clean sequence to remove mod masses and change oxMet to 'm'
    fraglist = []
    z=1
    for x in range(1, len(sequence)-1):
        fragseq = sequence[x:]
        lightfragmz = mass.calculate_mass(sequence=sequence[x:], ion_type='y', charge=1, aa_comp = customAAcomp)
        ## part to count the number of K and R residues
        nk = fragseq.count('K')
        nr = fragseq.count('R')
        heavyfragmz = lightfragmz + (hK/z)*nk + (hR/z)*nr
        fraglist.append([fragseq, lightfragmz, heavyfragmz])
    fl = pd.DataFrame(fraglist)
    fl.columns = ['fragseq', 'lightmz', 'heavymz']
    predy = round(fl['lightmz'],ndigits = 3).tolist()
    foundmzs = [libmzarray[y] for y in [libmzarray.index(x) for x in predy if x in libmzarray]]
    foundints = [libintarray[y] for y in [libmzarray.index(x) for x in predy if x in libmzarray]]
    keeplight = pd.DataFrame([foundmzs, foundints]).T.sort_values(by=1, ascending=False)[0][:3].tolist()
    lightmzs = round(fl['lightmz'], ndigits=3).tolist()
    fl = fl.loc[[lightmzs.index(x) for x in keeplight]]
    fl['ordinal'] = [len(x) for x in fl['fragseq'].tolist()]
    return(fl)


### Loop through the table of peptides, add their quant fragments to the list

In [24]:
#speclib = mgf.read('P:/JGM_DI2A/MSPLIT-DIAv1.0/human.faims.fixed.decoy.mgf')
y_light_quantfrags = []
y_heavy_quantfrags = []
ordinals = []
for i in range(len(sortedpepdf['Peptide'])):
    SEQ = sortedpepdf['Peptide'][i]
    SPEC =  sortedpepdf['Name'][i]
    if(SPEC!='DECOY_null'):
        tmpfrags = getQuantFragsFromLib(SEQ, SPEC, speclib)
        y_light_quantfrags.append([round(x, 4) for x in tmpfrags['lightmz'].tolist()])
        y_heavy_quantfrags.append([round(x, 4) for x in tmpfrags['heavymz'].tolist()])
        ordinals.append(tmpfrags['ordinal'].tolist())
    if(SPEC=='DECOY_null'):
        y_light_quantfrags.append('NA')
        y_heavy_quantfrags.append('NA')
        ordinals.append('NA')


In [25]:
sortedpepdf['ylight_quant_frags'] = y_light_quantfrags
sortedpepdf['yheavy_quant_frags'] = y_heavy_quantfrags
sortedpepdf['fragment_ordinals'] = ordinals


In [26]:
sortedpepdf.head()

,level_0,index,#File,Scan#,Mz,z,Peptide,Mz.1,z.1,cosine,...,stat17,stat18,stat19,CV,Heavymz,quantprecursor,quantscans,ylight_quant_frags,yheavy_quant_frags,fragment_ordinals
0,0,3112,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,2420,420.0,2,APIIAVTR,420.77,2,0.976826,...,2.925132,NaN,NaN,-70.0,425.77,420.5,"[1618, 4024]","[559.3562, 446.2722, 672.4403]","[569.3645, 456.2804, 682.4486]","[5, 4, 6]"
1,1,166,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,403,803.0,2,ALTGHLEEVVLALLK,803.48,2,0.973920,...,0.161017,NaN,NaN,-30.0,807.49,803.0,"[269, 2675]","[260.1969, 1013.6241, 1126.7082]","[268.2111, 1021.6383, 1134.7224]","[2, 9, 10]"
2,2,123,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,375,775.0,2,GTDVNVFNTILTTR,775.91,2,0.972525,...,0.220307,NaN,NaN,-30.0,780.91,776.0,"[251, 2657]","[965.5415, 1178.6528, 818.473]","[975.5497, 1188.6611, 828.4813]","[8, 10, 7]"
3,3,85,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,333,733.0,2,S+42.01057DAAVDTSSEITTK,733.85,2,0.970295,...,0.807127,NaN,NaN,-30.0,737.86,734.0,"[223, 2629]","[981.4735, 1080.5419, 349.2082]","[989.4877, 1088.5561, 357.2224]","[9, 10, 3]"
4,4,744,P:\JGM_DI2A\SILAC\20190411_DI2A_1to16_n1b.mzXML,785,585.0,2,ILLAELEQLK,585.36,2,0.970021,...,0.036934,NaN,NaN,-40.0,589.37,585.5,"[525, 2931]","[943.5459, 830.4618, 759.4247]","[951.5601, 838.476, 767.4389]","[8, 7, 6]"


### Write table for import into R

In [110]:
sortedpepdf.to_csv("P:/JGM_DI2A/Python/outputs/peptide_target_table_unfiltered.txt", sep="\t")

## Reason it moves to R here is I don't know how to match peptides to proteins in python

### use R to generate lists of targets to make instrument data collection method
but also to make sublist of selected peptides to quantify for each protein based on the most intense peptide observed